In this notebook we'll flesh our where we can abstract things away (into their own files), improve the extensibility and add some neat things to improve our ability to prototype (like weights and biases)

In [1]:
from fastai.vision.all import *
from utils import *

In [3]:
train_df, sub_df = get_dfs()
dls = get_dls(train_df, bs=64);
#dls.show_batch(max_n=6);

In [4]:
model_name = 'convnext_base'
learner = create_learner(dls, model_name=model_name).to_fp16()

In [5]:
learner.fine_tune(10)

epoch,train_loss,valid_loss,accuracy,time
0,1.021924,0.717575,0.763095,00:56


epoch,train_loss,valid_loss,accuracy,time
0,0.330589,0.283131,0.913023,00:56
1,0.246656,0.222099,0.937050,00:58
2,0.256539,0.260480,0.915906,00:56
3,0.193060,0.184519,0.942335,00:56
4,0.152685,0.157338,0.956271,00:55
5,0.097100,0.106628,0.967323,00:56
6,0.067475,0.104174,0.969726,00:56
7,0.043214,0.071732,0.977895,00:56
8,0.025066,0.072480,0.978856,00:56
9,0.019073,0.071043,0.979337,00:56


In [8]:
learner.save('cnext_base_10ep');

In [6]:
test_dl = learner.dls.test_dl(sub_df)

In [11]:
tta_preds = learner.tta(dl=test_dl)

In [12]:
#test_preds = learner.get_preds(dl=test_dl)

In [17]:
len(sub_df), tta_preds[0].shape, tta_preds[1]

(3469, torch.Size([3469, 10]), None)

In [68]:
vocab = list(dls.vocab);
vocab_dict = {idx:lbl for idx,lbl in list(enumerate(vocab))}

In [71]:
sub_df['label'] = pd.Series(tta_preds[0].argmax(dim=1)).map(vocab_dict)
sub_df = sub_df[['image_id','label']]

In [72]:
sub_df.head(2)

,image_id,label
0,200001.jpg,hispa
1,200002.jpg,normal


In [73]:
sub_df.to_csv('data/submission.csv', index=False)

In [74]:
!kaggle competitions submit -c paddy-disease-classification -f data/submission.csv -m "0.979_10ep_224px_randomsplit"

100%|██████████████████████████████████████| 70.7k/70.7k [00:00<00:00, 93.5kB/s]
Successfully submitted to Paddy Doctor: Paddy Disease Classification

Nice! Somehow our random split of the data aligns well with LB score of: 0.97962. validation score on this run was 0.978856 in second to last epoch (it dropped in last).

Something interesting to note is that when using a stratified split, the validation scores are way worse. Does this represent data leakage or maybe the LB is just also sampled via a random split? -- maybe their random split leads to leakage as well?